This notebook will document what happens when executing conflicting_qwl.sh 


<h3>generate_network.py</h3>
The frist thing happening will be to generate the Network with parameters 20 0.5 1.3 and $h (6 10 ...)

Several parameters are changable by the user. These are:
Network Size,
Node Event Ratio,
Eventskew,
Number of Eventtypes

2. Eventrates will be created using numpy np.random.zipf

In [6]:
import numpy as np

print("Generating Eventrates")
print("----------------------")
eventrates = np.random.zipf(1.3, 6)
print(eventrates)
print("----------------------")

Generating Eventrates
----------------------
[629   3  10  14   1   1]
----------------------


Afterwards we will create our random hierarchical network tree using our node class.
1. Loop through 1 - Networksize
-> Random Computing power and Memory will be assigned to each network node
2. Each node gets assigned a random parent in the network and siblings are assigned
3. After all nodes are created and assigned we will add each leaf node (no childrens) a eventrate like following (random distribution for which events are generated)
-> The non Leaf nodes will receive an list with 0 * len(eventrates)


In [8]:
n_e_r = 0.5
myevents = []
for i in range(len(eventrates)):
    x = np.random.uniform(0,1)
    if x < n_e_r:
        myevents.append(eventrates[i])
    else:
        myevents.append(0)

print(myevents)

[0, 0, 10, 14, 0, 1]


After succsefull generation of Network Tree we create 2 binarie files:
- Create networkExperimentData
    - Array of Eventskew (1.3), number of eventtypes (6), node event ratio (0.5), Min(eventrates)/Max(eventrates)
- Create network
    - save the network array of nodes 

<h3>generate_graph.py</h3>

First we will use network binary and then create a networkX Graph
and then create a fog graph using the root node.
In this we only iterate from the root node to the bottom and add the the correct nodes and edges to one another


<h3>allpairs.py</h3>

in this code we take a look at all nodes and compute the shortest distance from each node to all nodes using the dijkstra algorithm

In [2]:
import pickle
import multiprocessing
import time
import networkx as nx 

with open('graph',  'rb') as graph_file:
    G = pickle.load(graph_file)

myNodes = list(G.nodes)
allPairs = [[] for x in myNodes]
print("Printing a list of empty distances between each node")
print("----------------")
print(allPairs)
print("----------------")

def fillMyMatrice(me):  
    myDistances = []
    for j in range(len(G.nodes)):            
           myDistances.append(len(nx.shortest_path(G, me, j, method='dijkstra')) - 1)   
    return (me, myDistances)

pool = multiprocessing.Pool()
         


start = time.time()
result = pool.map(fillMyMatrice, myNodes)
for i in result:
    allPairs[i[0]] = i[1]
end = time.time()
print("Here are the distances between all nodes to one another")
print("----------------")
print(allPairs)
print("----------------")

Printing a list of empty distances between each node
----------------
[[], [], [], [], [], [], [], [], [], []]
----------------
Here are the distances between all nodes to one another
----------------
[[0, 1, 1, 1, 2, 2, 2, 1, 1, 2], [1, 0, 2, 2, 1, 3, 1, 2, 2, 3], [1, 2, 0, 2, 3, 3, 3, 2, 2, 3], [1, 2, 2, 0, 3, 1, 3, 2, 2, 3], [2, 1, 3, 3, 0, 4, 2, 3, 3, 4], [2, 3, 3, 1, 4, 0, 4, 3, 3, 4], [2, 1, 3, 3, 2, 4, 0, 3, 3, 4], [1, 2, 2, 2, 3, 3, 3, 0, 2, 1], [1, 2, 2, 2, 3, 3, 3, 2, 0, 3], [2, 3, 3, 3, 4, 4, 4, 1, 3, 0]]
----------------


Next we need to generate the Query workload 

<h3>generate_qwls.py</h3>

First we call the generate_workload function with count=3 and length=5
It creates random queries between SEQ and AND operations.
The children being the Primitive Events or subqueries

In [29]:
with open('current_wl', 'rb') as current_wl:
    cwl = pickle.load(current_wl)
    print(cwl)
    for i in cwl:
        print(i.children)

[<tree.SEQ object at 0x7f6cf82abdf0>, <tree.AND object at 0x7f6cf9d54ca0>, <tree.AND object at 0x7f6ce1b73a00>]
[<tree.PrimEvent object at 0x7f6cf9d5d4c0>, <tree.PrimEvent object at 0x7f6cf8338340>, <tree.PrimEvent object at 0x7f6cd0386850>]
[<tree.PrimEvent object at 0x7f6cf8285580>, <tree.SEQ object at 0x7f6cf8398460>]
[<tree.PrimEvent object at 0x7f6ce1b73a60>, <tree.SEQ object at 0x7f6ce1b73cd0>]


After generating the Queries we dump into into a binary file
- Create current_wl

<h3>generate_selectivity.py</h3>
generates selectivities for a given tuple of primitve event types
first uses the network binary to create a List of primitive events (Letters)
the following way:

In [14]:
import string

with open('network',  'rb') as  nw_file:
        nw = pickle.load(nw_file)
    
PrimitiveEvents = list(string.ascii_uppercase[:3])

print(nw[5].eventrates)
print(PrimitiveEvents)

[0, 7, 0, 0, 73, 11, 4, 0, 0, 0, 0, 0, 0, 0, 0, 99071, 0, 66, 17, 11]
['A', 'B', 'C']


After initilization of Primitive Eventtypes the selectivities for each tuple is created. A function receives the list of primitive Events and 2 parameters
x = 0.1 and y=0.01
1. Generate all possible combinations of Primitive Events
2. assign selectivities to each pair of Primitive Events
    - selectivities are calculated randomly either as 1 or a random number calculated through random.uniform(0.1,0.01)

In [16]:
from helper import generate_twosets
print(generate_twosets(PrimitiveEvents))

import random as rd

print(rd.uniform(0.1,0.01))
#initialize_selectivities(PrimitiveEvents,0.1,0.01)

['AC', 'AB', 'CB']
0.03937781862900642


After generating all selectivities 
- Create selectivitiesExperimentData
    - Array of [0.1 and median of all selectivities]
- Create selectivities
    - Dumping the dictionary of all selectivities into a file
    

<h3>write_config_single.py</h3>

writes into a .txt file
- Network structure
- Query workload
- Example MuSE graph query
- Selectivitiy 

<h3>determine_all_single_selectivities.py</h3>

1. Opens the config_single_selectivity.txt
2. Opens the Workload (Queries)
3. Iterates over the Queries and generates selectivities for each Primitive Events and the corresponding queries (for example SEQ(P,I,O))
4. creates singleSelectivities binary file with these selectivities of projections

<h3>gemerate_projections.py</h3>

1. Iterates over the workload binaries (current_wl) and generates beneficial for each projection
    - 

In [40]:
import pickle
with open('current_wl','rb') as queries:
    cwl = pickle.load(queries)
    # print(cwl)
    # for i in cwl:
    #     print(i.children)
from generate_projections import generate_projections

my_quer = cwl[0]
print(f"Query: {my_quer}")
prim_events = my_quer.children

for i in prim_events:
    print(i)

result = generate_projections(cwl[0])
for i in result:
    for j in i:
        print(j)    
        try:
            print(i[j])
        except:
            continue

Query: SEQ(P, I, O)
P
I
O
SEQ(I, O)
SEQ(P, I, O)
SEQ(I, O)
(0.08071676233790777, 4.681572215598651)
SEQ(P, I, O)
(0.005948036949143954, 34178.12217814116)


First all Queries are iterated over then possible projections are generated. Each projection gets attached a selectivity and a rate!

<h3>combigen.py</h3>

This is where the optimizing happens of the queries

Binaries are created
- Creates curcombi
- Creates originalCombiDict
- Creates criticalMSTypes
- Creates filterDict

In [58]:
import pickle
from generate_projections import projsPerQuery

with open('current_wl','rb') as workload:
    query = pickle.load(workload)

print(query[0])
from combigen import MSoptionsPerEvent, extractMsOptions, getBestChainCombis, promisingChainProjection
msoptions = MSoptionsPerEvent(query[0])
extracted = extractMsOptions(query[0])
promisingChain = promisingChainProjection(query[0])
print(promisingChain)
print(extracted)



SEQ(P, I, O)
{}
['P']


Curcombi will be my combination of projections

In [84]:
import pickle
from placement_aug import NEWcomputeCentralCosts

with open('current_wl', 'rb') as workload:
    wl = pickle.load(workload)
costs = NEWcomputeCentralCosts(wl)
print(costs)

(991975.0, 0, 2, {'P': {'P2': [2, 0], 'P4': [4, 1, 0], 'P5': [5, 3, 0], 'P6': [6, 1, 0], 'P8': [8, 0], 'P9': [9, 7, 0]}, 'I': {'I6': [6, 1, 0]}, 'O': {'O2': [2, 0], 'O4': [4, 1, 0], 'O8': [8, 0], 'O9': [9, 7, 0]}, 'G': {'G2': [2, 0], 'G4': [4, 1, 0], 'G5': [5, 3, 0], 'G6': [6, 1, 0], 'G8': [8, 0]}, 'B': {'B5': [5, 3, 0], 'B6': [6, 1, 0], 'B8': [8, 0]}, 'J': {'J2': [2, 0]}, 'E': {'E5': [5, 3, 0], 'E6': [6, 1, 0], 'E9': [9, 7, 0]}, 'Q': {'Q6': [6, 1, 0], 'Q9': [9, 7, 0]}, 'R': {'R5': [5, 3, 0]}, 'H': {'H2': [2, 0], 'H4': [4, 1, 0], 'H6': [6, 1, 0]}})


In [67]:
import pickle
with open('curcombi', 'rb') as curcombi:
    cc = pickle.load(curcombi)
    print(cc)
print(len(cc))
for key, value in cc.items():
    print(f"Key: {key}, value: {value}")

{<tree.SEQ object at 0x7f739aac1220>: ['P', <tree.SEQ object at 0x7f739a888dc0>], <tree.SEQ object at 0x7f739a888dc0>: ['I', 'O'], <tree.AND object at 0x7f7393f1adf0>: ['G', 'P', 'B', 'J'], <tree.AND object at 0x7f7393f1a1c0>: ['P', <tree.SEQ object at 0x7f7393f1a0d0>], <tree.SEQ object at 0x7f7393f1a0d0>: ['R', <tree.SEQ object at 0x7f7393fa6550>], <tree.SEQ object at 0x7f7393fa6550>: [<tree.AND object at 0x7f7393fa60d0>, 'E'], <tree.AND object at 0x7f7393fa60d0>: ['Q', 'H']}
7
Key: SEQ(P, I, O), value: ['P', <tree.SEQ object at 0x7f739a888dc0>]
Key: SEQ(I, O), value: ['I', 'O']
Key: AND(G, SEQ(P, B, J)), value: ['G', 'P', 'B', 'J']
Key: AND(P, SEQ(E, AND(Q, R, H))), value: ['P', <tree.SEQ object at 0x7f7393f1a0d0>]
Key: SEQ(E, AND(Q, R, H)), value: ['R', <tree.SEQ object at 0x7f7393fa6550>]
Key: SEQ(E, AND(Q, H)), value: [<tree.AND object at 0x7f7393fa60d0>, 'E']
Key: AND(Q, H), value: ['Q', 'H']


In [68]:
import pickle
with open('originalCombiDict', 'rb') as curcombi:
    cc = pickle.load(curcombi)
    print(cc)
for key, value in cc.items():
    print(f"Key: {key}, value: {value}")

{<tree.AND object at 0x7f73981f28e0>: (['P', 'H'], ['P', 9], 707228.5615905413), <tree.SEQ object at 0x7f739aac1bb0>: (['E', 'Q'], ['E', 6], 53.54975153928859), <tree.SEQ object at 0x7f7393f24430>: (['E', 'H'], ['E', 6], 351.0894377669257), <tree.AND object at 0x7f7393f24ac0>: (['Q', 'H'], ['Q', 4], 7.048862838444904), <tree.AND object at 0x7f7393f24700>: ([<tree.SEQ object at 0x7f7393f24430>, 'P'], ['P'], 926261.6482890521), <tree.AND object at 0x7f7393f24730>: ([<tree.AND object at 0x7f7393f24ac0>, 'P'], ['P'], 1204884.7378928768), <tree.SEQ object at 0x7f7393f243a0>: ([<tree.AND object at 0x7f7393f24ac0>, 'E'], ['E'], 444.00252888456055), <tree.AND object at 0x7f7398005f40>: (['P', <tree.SEQ object at 0x7f7393f243a0>], ['P', 9], 1247157.392315714), <tree.SEQ object at 0x7f7393fa6910>: (['R', <tree.SEQ object at 0x7f7393f243a0>], ['R', 0], 555.1012260658636), <tree.AND object at 0x7f7393fa6280>: (['P', <tree.SEQ object at 0x7f7393fa6910>], ['P', 9], 1248735.1124303725), <tree.SEQ obj

<h3>computePlanCosts_aug.py</h3>

This file calculates all costs for all the binary files
1. Starts off by Computing Central Costs for the Workload

In [69]:
import pickle

with open('current_wl', 'rb') as workload:
    wl = pickle.load(workload)
    
for i in wl:
    print(i.leafs())

['P', 'I', 'O']
['G', 'P', 'B', 'J']
['P', 'E', 'Q', 'R', 'H']
